In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import pandas_profiling
from pyod.models.knn import KNN

In [2]:
data_path = 'as_is.csv'
df = pd.read_csv(data_path)

In [3]:
profile = df.profile_report(title='EDA Profile')
profile.to_file(output_file="bgp_eda.html")

In [4]:
cols = list(df.columns)

df_normal = df.loc[df["Anomalous"] == -1]
df_anom   = df.loc[df["Anomalous"] ==  1]

df_train = df.sample(frac=0.8,random_state=1)
df_test = df.drop(df_train.index)

x_train = df_train[cols[0:37]]
y_train = df_train["Anomalous"]
y_train.loc[y_train == -1] = 0

x_test = df_test[cols[0:37]]
y_test = df_test["Anomalous"]
y_test.loc[y_test == -1] = 0



F:\Anaconda3\envs\bgp\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [5]:
#Compute contimination of anomamlies
n_norm = sum(df['Anomalous'] == -1)
n_anom = sum(df['Anomalous'] ==  1)
cont_factor = n_anom / (n_norm + n_anom)
print('Contamination Factor for whole data set: ',cont_factor * 100 , "%")

n_norm_train = sum(y_train == 0)
n_anom_train = sum(y_train ==  1)
cont_factor_train = n_anom_train / (n_norm_train + n_anom_train)
print('Contamination Factor for training set: ',cont_factor_train * 100 , "%")


Contamination Factor for whole data set:  16.513427702076992 %
Contamination Factor for training set:  16.544710405556245 %


In [6]:
clf_name = 'KNN'
clf = KNN(contamination=cont_factor_train)
clf.fit(x_train.to_numpy())

KNN(algorithm='auto', contamination=0.16544710405556246, leaf_size=30,
  method='largest', metric='minkowski', metric_params=None, n_jobs=1,
  n_neighbors=5, p=2, radius=1.0)

In [7]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

In [8]:
# get the prediction on the test data
y_test_pred = clf.predict(x_test.to_numpy())  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(x_test.to_numpy())  # outlier scores

In [9]:
from pyod.utils.data import evaluate_print
# evaluate and print the results
print("\nOn Training Data:")
evaluate_print(clf_name, y_train.to_numpy(), y_train_scores)
print("\nOn Test Data:")
evaluate_print(clf_name, y_test.to_numpy(), y_test_scores)


On Training Data:
KNN ROC:0.6373, precision @ rank n:0.2046

On Test Data:
KNN ROC:0.633, precision @ rank n:0.1937
